In [1]:
import pandas as pd # standard python data library
import geopandas as gp # the geo-version of pandas
import numpy as np 
import os
import fiona
from statistics import mean, median
from pandas import read_csv
gp.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw' #To load KML files
import string
import xml.etree.ElementTree as et

# Georgia

## Election Results Documentation:

Election results from the Georgia Secretary of State Elections Division (https://sos.ga.gov/index.php/Elections/current_and_past_elections_results). Presidential recount results from the Georgia Secretary of State Elections Division via Reuters.

## Shapefile Source Documentation:

Precinct shapefile primarily from the Georgia General Assembly Reapportionment Office (http://www.legis.ga.gov/Joint/reapportionment/en-US/default.aspx).  
Cobb, DeKalb, and Gwinnett counties instead use shapefiles from the U.S. Census Bureau's 2020 Redistricting Data Program.  
Forsyth and Fulton use shapefiles sourced from the respective counties.

## Shapefile Modifications Documentation:

Three of the four VTDs in Chattahoochee County are comprised of Fort Benning. However, the county only reports one polling location for all voters, including residents of Fort Benning that vote within the county. The four Chattahoochee County VTDs have therefore been merged in the shapefile.

The following additional modifications reflect changes made prior to the 2020 general election.

Barrow: Merge 2/15, 3/12, 4/14, 5/7, 6/10/13, 8/9, 11/16; Adjust new 2/13 boundary  
Bartow: Split Cassville/Hamilton Crossing  
Candler: Merge Candler/Metter as Jack Strickland Comm Center  
Chatham: Split 7-7/8-16, 7-12/7-16; Realign 7-06C/7-07C  
Chatooga: Split Cloudland/Teloga along ridgeline that marks boundary between them with the USGS Topographic Contour shapefile  
Clayton: Split Ellenswood 1/2, Jonesboro 1/17/19, Lovejoy 3/6/7, Morrow 3/11, 5/10, Oak 3/5   
Cobb: Split Bells Ferry 3/4, Dobbins 1/2, Marietta 3A/3B, Smyrna 3A/3B  
Columbia: Split Bessie Thomas/2nd Mt Moriah, Harlem Branch/Harlem Senior Ctr; Merge Blanchard Park/MTZ Col FD;  Align multiple precincts with county maps  
Coweta: Merge Arts Centre/Jefferson Parkway as Newnan Centre  
Fulton: Merge CP07A/CP07D, CH01/CH04B, SS29A/SS29B, UC031/UC035  
DeKalb: Split Clarkston/Clarkston Comm Ctr; Realign Decatur/Oakhurst; Align precincts with Atlanta, Brookhaven, Decatur, Tucker city limits   
Gwinnett: Adjust Baycreek F/G, Berkshire J/M, Cates D/F, Garners C/B, Lawrenceville G/N, Pinckneyville S/T, Rockbridge A/G  
Lowndes: Split Northgate Assembly/Trinity, Jaycee/Mt Calvary/Northside/VSU  
Oconee: Merge Annex/City Hall; Align City Hall with Watkinsville city limits  
Paulding: Reorganize 12 precincts into 19 precincts as redrawn in 2019  
Randolph: Merge Carnegie/Cuthbert-Courthouse, 4th District/Fountain Bridge/Shellman  
Troup: Split Mountville between Gardner Newman/Hogansville/Rosemont; Align multiple precincts with county maps  
Towns: Merge Macedonia/Tate City  
Wilkes: Align 1/2A boundary with the voter file    
Note that the leading zeros in the Paulding County precinct IDs are included in some election reports and omitted in others. The shapefile includes the leading zeros consistent with the voter file.

## Candidate List

See the README for a full candidate list

### Load Non-Recount Election Data

This data had to be downloaded county-by-county in XML format. The below code parses the XML and grabs the necessary data, adds it to a list, gives it the appropriate column names, and converts the data into a dataframe.

In [2]:
loaded_counties = os.listdir("./raw-from-source/official_county_results/")
z=[]
for locale in loaded_counties:
    print(locale)
    if locale.endswith('.xml'):
        file_string = "./raw-from-source/official_county_results/"+locale
        xtree = et.parse(file_string)
        xroot = xtree.getroot()
        store_list = []
        county_area = xroot.findall(".//Region")
        for i in county_area:
            county = i.text
        contests = xroot.findall(".//Contest")
        for i in contests:
            contest = i.attrib.get('text')
            if locale == "Primary.xml":
                contest += ("-(06/09)")
            elif locale == "Runoff.xml":
                contest += ("-(08/11)")
            else:
                raise ValueError
            lower = i.findall("./Choice")
            for j in lower:
                choice = j.attrib.get('text')
                lower_2 = j.findall("./VoteType")
                for k in lower_2:
                    voting_method = k.attrib.get('name')
                    lower_3 = k.findall("./County")
                    for l in lower_3:
                        precinct_name = l.attrib.get('name')
                        num_votes = l.attrib.get('votes')
                        z.append([county,contest,choice,voting_method,precinct_name,num_votes,locale])
dfcols = ['state','contest','choice','voting_method','county','num_votes',"file"]
ga_election = pd.DataFrame(z,columns=dfcols)

.DS_Store
Runoff.xml
detailxml (9).zip
detailxml (10).zip
Primary.xml


In [3]:
ga_election.drop(["state","file"], inplace = True, axis = 1)

In [4]:
ga_election["contest"] = ga_election["contest"].str.strip()

In [5]:
# Standardize the contest names
contest_name_changes = {
 'REP - US House Dist 4/Distrito del Congreso 4-(06/09)':'REP - US House Dist 4-(06/09)',
 'DEM - US House Dist 4/Distrito del Congreso 4-(06/09)':'DEM - US House Dist 4-(06/09)',
 'REP - US House Dist 7/Distrito del Congreso 7-(06/09)':'REP - US House Dist 7-(06/09)',
 'DEM - US House Dist 7/Distrito del Congreso 7-(06/09)':'DEM - US House Dist 7-(06/09)',
 'REP - US House Dist 10/Distrito del Congreso 10-(06/09)':'REP - US House Dist 10-(06/09)',
 'DEM - US House Dist 10/Distrito del Congreso 10-(06/09)':'DEM - US House Dist 10-(06/09)',
 'DEM - State House Dist 81/Estatal ante la Asamblea General Dist 81-(06/09)':'DEM - State House Dist 81-(06/09)',
 'REP - State House Dist 93/Estatal ante la Asamblea General Dist 93-(06/09)':'REP - State House Dist 93-(06/09)',
 'DEM - State House Dist 93/Estatal ante la Asamblea General Dist 93-(06/09)':'DEM - State House Dist 93-(06/09)',
 'DEM - State House Dist 94/Estatal ante la Asamblea General Dist 94-(06/09)':'DEM - State House Dist 94-(06/09)',
 'REP - State House Dist 95/Estatal ante la Asamblea General Dist 95-(06/09)':'REP - State House Dist 95-(06/09)',
 'DEM - State House Dist 95/Estatal ante la Asamblea General Dist 95-(06/09)':'DEM - State House Dist 95-(06/09)',
 'DEM - State House Dist 96/Estatal ante la Asamblea General Dist 96-(06/09)':'DEM - State House Dist 96-(06/09)',
 'REP - State House Dist 97/Estatal ante la Asamblea General Dist 97-(06/09)':'REP - State House Dist 97-(06/09)',
 'DEM - State House Dist 97/Estatal ante la Asamblea General Dist 97-(06/09)':'DEM - State House Dist 97-(06/09)',
 'REP - State House Dist 98/Estatal ante la Asamblea General Dist 98-(06/09)':'REP - State House Dist 98-(06/09)',
 'DEM - State House Dist 98/Estatal ante la Asamblea General Dist 98-(06/09)':'DEM - State House Dist 98-(06/09)',
 'DEM - State House Dist 99/Estatal ante la Asamblea General Dist 99-(06/09)':'DEM - State House Dist 99-(06/09)',
 'DEM - State House Dist 100/Estatal ante la Asamblea General Dist 100-(06/09)':'DEM - State House Dist 100-(06/09)',
 'REP - State House Dist 101/Estatal ante la Asamblea General Dist 101-(06/09)':'REP - State House Dist 101-(06/09)',
 'DEM - State House Dist 101/Estatal ante la Asamblea General Dist 101-(06/09)':'DEM - State House Dist 101-(06/09)',
 'REP - State House Dist 102/Estatal ante la Asamblea General Dist 102-(06/09)':'REP - State House Dist 102-(06/09)',
 'DEM - State House Dist 102/Estatal ante la Asamblea General Dist 102-(06/09)':'DEM - State House Dist 102-(06/09)',
 'REP - State House Dist 103/Estatal ante la Asamblea General Dist 103-(06/09)':'REP - State House Dist 103-(06/09)',
 'DEM - State House Dist 103/Estatal ante la Asamblea General Dist 103-(06/09)':'DEM - State House Dist 103-(06/09)',
 'REP - State House Dist 104/Estatal ante la Asamblea General Dist 104-(06/09)':'REP - State House Dist 104-(06/09)',
 'DEM - State House Dist 104/Estatal ante la Asamblea General Dist 104-(06/09)':'DEM - State House Dist 104-(06/09)',
 'REP - State House Dist 105/Estatal ante la Asamblea General Dist 105-(06/09)':'REP - State House Dist 105-(06/09)',
 'DEM - State House Dist 105/Estatal ante la Asamblea General Dist 105-(06/09)':'DEM - State House Dist 105-(06/09)',
 'REP - State House Dist 106/Estatal ante la Asamblea General Dist 106-(06/09)':'REP - State House Dist 106-(06/09)',
 'DEM - State House Dist 106/Estatal ante la Asamblea General Dist 106-(06/09)':'DEM - State House Dist 106-(06/09)',
 'REP - State House Dist 107/Estatal ante la Asamblea General Dist 107-(06/09)':'REP - State House Dist 107-(06/09)',
 'DEM - State House Dist 107/Estatal ante la Asamblea General Dist 107-(06/09)':'DEM - State House Dist 107-(06/09)',
 'REP - State House Dist 108/Estatal ante la Asamblea General Dist 108-(06/09)':'REP - State House Dist 108-(06/09)',
 'DEM - State House Dist 108/Estatal ante la Asamblea General Dist 108-(06/09)':'DEM - State House Dist 108-(06/09)',
 'REP - State House Dist 114/Estatal ante la Asamblea General Dist 114-(06/09)':'REP - State House Dist 114-(06/09)',
 'DEM - State Senate Dist 5/Senador Estatal Dist 5-(06/09)':'DEM - State Senate Dist 5-(06/09)',
 'REP - State Senate Dist 40/Senador Estatal Dist 40-(06/09)':'REP - State Senate Dist 40-(06/09)',
 'DEM - State Senate Dist 40/Senador Estatal Dist 40-(06/09)':'DEM - State Senate Dist 40-(06/09)',
 'REP - State Senate Dist 41/Senador Estatal Dist 41-(06/09)':'REP - State Senate Dist 41-(06/09)',
 'DEM - State Senate Dist 41/Senador Estatal Dist 41-(06/09)':'DEM - State Senate Dist 41-(06/09)',
 'REP - State Senate Dist 45/Senador Estatal Dist 45-(06/09)':'REP - State Senate Dist 45-(06/09)',
 'DEM - State Senate Dist 45/Senador Estatal Dist 45-(06/09)':'DEM - State Senate Dist 45-(06/09)',
 'REP - State Senate Dist 48/Senador Estatal Dist 48-(06/09)':'REP - State Senate Dist 48-(06/09)',
 'DEM - State Senate Dist 48/Senador Estatal Dist 48-(06/09)':'DEM - State Senate Dist 48-(06/09)',
 'DEM - State Senate Dist 55/Senador Estatal Dist 55-(06/09)':'DEM - State Senate Dist 55-(06/09)',
 'REP - President of the United States/Presidentede los Estados Unidos-(06/09)':'REP - President of the United States-(06/09)',
 'DEM - President of the United States/Presidentede los Estados Unidos-(06/09)':'DEM - President of the United States-(06/09)',
 'REP - US Senate/Senado de los EE.UU.-(06/09)':'REP - US Senate-(06/09)',
 'DEM - US Senate/Senado de los EE.UU.-(06/09)':'DEM - US Senate-(06/09)',
 'REP - Public Service Commission Dist 1/Comisionado de Servicio Público Dist 1-(06/09)':'REP - Public Service Commission Dist 1-(06/09)',
 'DEM - Public Service Commission Dist 1/Comisionado de Servicio Público Dist 1-(06/09)':'DEM - Public Service Commission Dist 1-(06/09)',
 'REP - Public Service Commission Dist 4/Comisionado de Servicio Público Dist 4-(06/09)':'REP - Public Service Commission Dist 4-(06/09)',
 'DEM - Public Service Commission Dist 4/Comisionado de Servicio Público Dist 4-(06/09)':'DEM - Public Service Commission Dist 4-(06/09)',
}

ga_election["contest"] = ga_election["contest"].map(contest_name_changes).fillna(ga_election["contest"])
ga_election["contest"] = ga_election["contest"].str.replace("District", "Dist")

In [6]:
ga_election["choice"] = ga_election["choice"].str.strip()

In [7]:
no_party = [i for i in list(ga_election["contest"].unique()) if "DEM -" not in i and "REP - " not in i]

In [8]:
ga_election["contest"] = np.where(ga_election["choice"].isin(['Scott Bohlke', 'Billy Hickman', 'Kathy Palmer','Neil Singleton']), "REP - "+ga_election["contest"], ga_election["contest"])
ga_election["contest"] = np.where(ga_election["choice"].isin(['Stephen Jared Sammons']), "IND - "+ga_election["contest"], ga_election["contest"])

In [9]:
no_party = [i for i in list(ga_election["contest"].unique()) if "DEM -" not in i and "REP - " not in i and "IND -" not in i]

In [10]:
total_races_list = list(ga_election["contest"].unique())

possible_us_house_races = [i for i in total_races_list if "US House" in i]
possible_ga_house_races = [i for i in total_races_list if "State House" in i]
possible_ga_senate_races = [i for i in total_races_list if "State Senate" in i]
possible_president_races = [i for i in total_races_list if "President of" in i]
possible_us_senate_races = [i for i in total_races_list if "US Senate" in i]
possible_possible_service_races = [i for i in total_races_list if "Public Service" in i]

In [11]:
# Make a list of all the above races
combined_races = possible_us_house_races + possible_ga_house_races + possible_ga_senate_races + possible_president_races +possible_us_senate_races + possible_possible_service_races

In [12]:
# # The code below looks at the races not included
# not_included_races = [i for i in total_races_list if i not in combined_races]
# not_included_races.sort()
# not_included_races

In [13]:
# Filter down to the needed races and clean indices
ga_election = ga_election[ga_election["contest"].isin(combined_races)]
ga_election.reset_index(inplace=True,drop=True)

### Clean and Pivot Data

In [14]:
ga_election["num_votes"] = ga_election["num_votes"].astype(int)

In [15]:
#ga_election["choice"] = ga_election["choice"].str.replace("Carden H. Summers", 'Carden H. Summers (I)') 
ga_election["choice"] = ga_election["choice"].str.replace("  ", ' ') 
ga_election["choice"] = ga_election["choice"].str.replace("Marcus A Wiedower", "Marcus A. Wiedower")

ga_election["choice"] = ga_election["choice"].map({'Carden H. Summers':'Carden H. Summers (I)'}).fillna(ga_election["choice"])

#### Add a FIPS Column

In [16]:
fips_file = pd.read_csv("./raw-from-source/FIPS/US_FIPS_Codes.csv")
fips_file = fips_file[fips_file["State"]=="Georgia"]
fips_file["FIPS County"] = fips_file["FIPS County"].astype(str)
fips_file["FIPS County"] = fips_file["FIPS County"].str.zfill(3)
fips_file["County Name"] = fips_file["County Name"].replace("De Kalb","DeKalb")
fips_dict = dict(zip(fips_file['County Name'], fips_file['FIPS County']))
ga_election['county_fips'] = ga_election['county'].map(fips_dict).fillna(ga_election['county'])
ga_election['county_fips'] = ga_election['county_fips'].astype(str)
ga_election['county_fips'] = ga_election['county_fips'].str.zfill(3)

#### Pivot the data

In [17]:
ga_election["unique_ID"]=ga_election["county_fips"]
ga_election["choice"]=ga_election["choice"]+"-:+:-"+ga_election["contest"]
ga_election=pd.pivot_table(ga_election,index=["county","county_fips"],columns=["choice"],values=['num_votes'],aggfunc=sum)
ga_election = ga_election.fillna(0)
ga_election.columns = ga_election.columns.droplevel(0)
ga_election = ga_election.reset_index()

In [18]:
ga_election

choice,county,county_fips,"""""Mokah"""" Jasmine Johnson-:+:-DEM - State House Dist 117-(06/09)",Afoma Eguh-Okafor-:+:-DEM - State Senate Dist 30-(06/09),Al Williams (I)-:+:-DEM - State House Dist 168-(06/09),Alan Cole-:+:-REP - State House Dist 80-(06/09),Alan Powell (I)-:+:-REP - State House Dist 32-(06/09),Alex B. Kaufman-:+:-REP - State House Dist 51-(06/09),Alex Wan-:+:-DEM - State House Dist 57-(06/09),Alfred Reynolds-:+:-DEM - State House Dist 93-(06/09),...,William Park Freeman-:+:-REP - State Senate Dist 41-(06/09),William Tauxe-:+:-DEM - State House Dist 134-(06/09),Winfred Dukes (I)-:+:-DEM - State House Dist 154-(06/09),Yasmin Neal-:+:-DEM - State House Dist 74-(06/09),Zachary H. Kennemore-:+:-REP - US House Dist 7-(06/09),Zachary Perry-:+:-DEM - State Senate Dist 46-(06/09),Zack Tumlin-:+:-REP - State House Dist 9-(06/09),Zahra S. Karinshak-:+:-DEM - US House Dist 7-(06/09),Zulma Lopez-:+:-DEM - State House Dist 86-(06/09),Zulma Lopez-:+:-DEM - State House Dist 86-(08/11)
0,Appling,001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Atkinson,003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Bacon,005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Baker,007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,637.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Baldwin,009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,Whitfield,313,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
155,Wilcox,315,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
156,Wilkes,317,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
157,Wilkinson,319,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Load in the Fields Dict

In [19]:
fields_dict_lower_df = pd.read_csv("./primary_fields_dict_full.csv")

In [25]:
fields_dict_lower_df[fields_dict_lower_df["Cand_Name"].str.contains("26")]

,Field_Name,Cand_Name
150,"David E. Lucas, Sr. (I)-:+:-DEM - State Senate...",PSU26DLUC
226,Gloria Frazier (I)-:+:-DEM - State House Dist ...,PSL126DFRA
254,Jason Boskey-:+:-DEM - State House Dist 26-(06...,PSL026DBOS
348,"Lauren W. McDonald, III-:+:-REP - State House ...",PSL026RMCD
597,Verbin Weaver-:+:-DEM - State Senate Dist 26-(...,PSU26DWEA


In [26]:
fields_dict_lower_df_dict = dict(zip(fields_dict_lower_df["Field_Name"], fields_dict_lower_df["Cand_Name"]))

In [27]:
ga_election.rename(columns = fields_dict_lower_df_dict, inplace = True)

In [28]:
ga_election

choice,county,county_fips,PSL117DJOH,PSU30DEGU,PSL168DWIL,PSL080RCOL,PSL032RPOW,PSL051RKAU,PSL057DWAN,PSL093DREY,...,PSU41RFRE,PSL134DTAU,PSL154DDUK,PSL074DNEA,PCON07RKEN,PSU46DPER,PSL009RTUM,PCON07DKAR,PSL086DLOP,RSL086DLOP
0,Appling,001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Atkinson,003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Bacon,005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Baker,007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,637.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Baldwin,009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,Whitfield,313,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
155,Wilcox,315,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
156,Wilkes,317,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
157,Wilkinson,319,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
ga_election.to_csv("./official_county_totals.csv", index = False)